<a href="https://colab.research.google.com/github/shashanksrajak/neural-networks-from-zero/blob/main/1_neural_net_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Neural Network from Scratch
The goal is to understand neural network and its inner working and then build one from scratch using all the mathematical concepts and tools.

In [2]:
import random
import math
import numpy as np

## Single Neuron

In [7]:
class Neuron:
  def __init__(self, nin):
    """
    A single neuron
    nin : number of inputs to this neuron
    """
    self.w = np.array([random.uniform(-1, 1) for _ in range(nin)]) # weights
    self.b = random.uniform(-1, 1) # bias term

  def __call__(self, x):
    wx = self.w * x
    z = sum(wx, self.b) # z = wx + b
    a = 1/(1+math.exp(-z))    # we are using sigmoid function but this can also be a user choice

    self.out = a
    return a

In [8]:
n = Neuron(3)

In [13]:
# sample input
x = np.array([random.randint(1, 10) for _ in range(3)])
x

array([ 5, 10,  1])

In [14]:
n(x)

0.9878307196697438

In [15]:
n.w

array([-0.77995667,  0.94358218, -0.87519861])

In [16]:
n.b

-0.26424324169995606

In [17]:
n.out

0.9878307196697438